# RibonanzaNet 3D Inference

In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import pickle

In [2]:
train_data=pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_sequences.csv")
train_labels=pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_labels.csv")

In [3]:
from torch.utils.data import Dataset, DataLoader

class RNADataset(Dataset):
    def __init__(self,data):
        self.data=data
        self.tokens={nt:i for i,nt in enumerate('ACGU')}

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sequence=[self.tokens[nt] for nt in (self.data.loc[idx,'sequence'])]
        sequence=np.array(sequence)
        sequence=torch.tensor(sequence)

        return {'sequence':sequence}

In [4]:
train_dataset=RNADataset(train_data)
train_dataset[0]

{'sequence': tensor([2, 2, 2, 3, 2, 1, 3, 1, 0, 2, 3, 0, 1, 2, 0, 2, 0, 2, 2, 0, 0, 1, 1, 2,
         1, 0, 1, 1, 1])}

## RhoFold Inference

In [5]:
import pandas as pd
import numpy as np
import os
import sys
import argparse

In [6]:
!pip install /kaggle/input/openmm/OpenMM-8.2.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl

Processing /kaggle/input/openmm/OpenMM-8.2.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl


In [7]:
!pip install /kaggle/input/simtk-0-1/simtk-0.1.0-py2.py3-none-any.whl

Processing /kaggle/input/simtk-0-1/simtk-0.1.0-py2.py3-none-any.whl


In [8]:
!pip install /kaggle/input/pytest-runner/pytest_runner-6.0.1-py3-none-any.whl

Processing /kaggle/input/pytest-runner/pytest_runner-6.0.1-py3-none-any.whl


In [9]:
!pip install /kaggle/input/biopython/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Processing /kaggle/input/biopython/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


In [10]:
!pip install /kaggle/input/ml-collections/ml_collections-1.0.0-py3-none-any.whl

Processing /kaggle/input/ml-collections/ml_collections-1.0.0-py3-none-any.whl


In [11]:
!cp -R /kaggle/input/rhofold-repo /kaggle/working/

In [12]:
!mv rhofold-repo RhoFold

In [13]:

cd /kaggle/working/RhoFold

/kaggle/working/RhoFold


In [14]:
!python setup.py install

/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/dist.py:261: UserWarning: Unknown distribution option: 'tests_require'
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or ot

In [15]:
fasta = train_data

In [16]:
fasta

,target_id,sequence,temporal_cutoff,description,all_sequences
0,1SCL_A,GGGUGCUCAGUACGAGAGGAACCGCACCC,1995-01-26,"THE SARCIN-RICIN LOOP, A MODULAR RNA",>1SCL_1|Chain A|RNA SARCIN-RICIN LOOP|Rattus n...
1,1RNK_A,GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU,1995-02-27,THE STRUCTURE OF AN RNA PSEUDOKNOT THAT CAUSES...,>1RNK_1|Chain A|RNA PSEUDOKNOT|null\nGGCGCAGUG...
2,1RHT_A,GGGACUGACGAUCACGCAGUCUAU,1995-06-03,24-MER RNA HAIRPIN COAT PROTEIN BINDING SITE F...,>1RHT_1|Chain A|RNA (5'-R(P*GP*GP*GP*AP*CP*UP*...
3,1HLX_A,GGGAUAACUUCGGUUGUCCC,1995-09-15,P1 HELIX NUCLEIC ACIDS (DNA/RNA) RIBONUCLEIC ACID,>1HLX_1|Chain A|RNA (5'-R(*GP*GP*GP*AP*UP*AP*A...
4,1HMH_E,GGCGACCCUGAUGAGGCCGAAAGGCCGAAACCGU,1995-12-07,THREE-DIMENSIONAL STRUCTURE OF A HAMMERHEAD RI...,">1HMH_1|Chains A, C, E|HAMMERHEAD RIBOZYME-RNA..."
...,...,...,...,...,...
839,8T3E_EC,AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAAC...,2024-12-11,Hypomethylated yeast 80S bound with Taura synd...,>8T3E_1|Chain A[auth BA]|40S ribosomal protein...
840,8T3F_EC,AAACUCCAUGUAUUGGUUACCCAUCUGCAUCGAAAACUCUCCGAAC...,2024-12-11,Hypomethylated yeast 80S bound with Taura synd...,>8T3F_1|Chain A[auth BA]|40S ribosomal protein...
841,8XCC_B,GUGCUGCUGUCUCCCAGACGGGAGGCAGAACUGCACCUUCCAUCAG...,2024-12-11,"Cryo-EM structure of Cas12j19 (E100K), crRNA a...",>8XCC_1|Chain A|Cas12j19(E100K)|unclassified s...
842,8Z1G_T,GGUAAAAUGGCUGAGUGAAGCAUUGGACUGUAAAUCUAAAGACAGG...,2024-12-18,Cryo-EM structure of human ELAC2-pre-tRNA,>8Z1G_1|Chain A[auth T]|Homo sapiens mitochond...


In [17]:
!mkdir test_fasta
!mkdir out

### Step : Writing the RNA sequences in Fasta Format

In [18]:
from tqdm import tqdm

for target_id in tqdm(fasta["target_id"].values):
    with open(f"test_fasta/{target_id}.fasta", "w") as text_file:
        text_file.write(">100500\n")
        text_file.write(
            fasta.loc[fasta["target_id"] == target_id, ["sequence"]].to_string(index = False, header = False)
        )

100%|██████████| 844/844 [00:00<00:00, 863.61it/s]


In [19]:
from tqdm import tqdm
for target_id in tqdm(fasta["target_id"].values):
    try:
        os.makedirs(f"./out/{target_id}", exist_ok = True)
        
        a3m_file     = f"/kaggle/input/stanford-rna-3d-folding/MSA/{target_id}.MSA.fasta"
        fasta_file   = f"/kaggle/working/RhoFold/test_fasta/{target_id}.fasta"
        
        if len(open(fasta_file, 'r').read()) > 400:
            continue
            
        device       = "cpu" if len(open(fasta_file, 'r').read()) > 400 else "cuda:0"
        run_folding  = f"python inference.py --relax_steps 0 --input_fas {fasta_file} --input_a3m {a3m_file} --output_dir ./out/{target_id}/ --device {device} --ckpt ./pretrained/RhoFold_pretrained.pt"
        os.system(run_folding)
    except Exception as e:
        print(e)

100%|██████████| 844/844 [2:31:07<00:00, 10.74s/it]


In [20]:
def extract_c1_coordinates(pdb_file):
    # Create a PDB parser object
    parser = PDBParser()
    
    # Load the structure from the PDB file
    structure = parser.get_structure('RNA_structure', pdb_file)
    
    # Initialize a list to store the coordinates of C1' atoms
    c1_coordinates = []
    
    # Iterate over all models in the structure (usually only one model)
    for model in structure:
        # Iterate over all chains in the model
        for chain in model:
            # Iterate over all residues in the chain
            for residue in chain:
                # Check if the residue is an RNA nucleotide
                if residue.get_resname() in ['A', 'U', 'G', 'C']:
                    # Try to get the C1' atom
                    try:
                        c1_atom = residue['C1\'']
                        # Append the coordinates of the C1' atom to the list
                        c1_coordinates.append((residue.get_resname(), c1_atom.get_coord()))
                    except KeyError:
                        # If C1' atom is not found, skip this residue
                        print(f"C1' atom not found in residue {residue.get_resname()}{residue.get_id()[1]}")
    
    return c1_coordinates

In [21]:
from pathlib import Path
from Bio.PDB import PDBParser

In [22]:
subm = []
for idx, row in fasta.iterrows():
    target_id = row["target_id"]
    try:
        fasta_file = f"/kaggle/working/RhoFold/test_fasta/{target_id}.fasta"    
        if len(open(fasta_file, 'r').read()) > 400:
            continue
        pdb_file = f"/kaggle/working/RhoFold/out/{target_id}/unrelaxed_model.pdb"
        if not Path(pdb_file).exists():
            continue
        coords = extract_c1_coordinates(pdb_file)
        res_name = np.array([x[0] for x in coords])
        res_num = np.array(range(len(res_name))) + 1
        coords = np.array([x[1] for x in coords])

        res = pd.DataFrame({
            "ID" : [target_id + "_" + x for x in res_num.astype("str")],
            "resname": res_name,
            "resid": res_num,
            "x_1": coords[:, 0],
            "y_1": coords[:, 1],
            "z_1": coords[:, 2],
            "x_2": coords[:, 0],
            "y_2": coords[:, 1],
            "z_2": coords[:, 2],
            "x_3": coords[:, 0],
            "y_3": coords[:, 1],
            "z_3": coords[:, 2],
            "x_4": coords[:, 0],
            "y_4": coords[:, 1],
            "z_4": coords[:, 2],
            "x_5": coords[:, 0],
            "y_5": coords[:, 1],
            "z_5": coords[:, 2],
        })
        subm.append(res)
    except Exception as e:
        print(e)
        seq      = row["sequence"]
        res_name = [i for i in seq]
        res_num  = np.array(range(len(seq))) + 1
        res = pd.DataFrame({
            "ID" : [target_id + "_" + x for x in res_num.astype("str")],
            "resname": res_name,
            "resid": res_num,
            "x_1": 0.0,
            "y_1": 0.0,
            "z_1": 0.0,
            "x_2": 0.0,
            "y_2": 0.0,
            "z_2": 0.0,
            "x_3": 0.0,
            "y_3": 0.0,
            "z_3": 0.0,
            "x_4": 0.0,
            "y_4": 0.0,
            "z_4": 0.0,
            "x_5": 0.0,
            "y_5": 0.0,
            "z_5": 0.0,
        })
        subm.append(res)

submit_rhofold = pd.concat(subm, axis = 0)
submit_rhofold

Empty file.
Empty file.


,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,1SCL_A_1,G,1,-2.356,12.116000,0.453,-2.356,12.116000,0.453,-2.356,12.116000,0.453,-2.356,12.116000,0.453,-2.356,12.116000,0.453
1,1SCL_A_2,G,2,-5.678,8.330000,2.603,-5.678,8.330000,2.603,-5.678,8.330000,2.603,-5.678,8.330000,2.603,-5.678,8.330000,2.603
2,1SCL_A_3,G,3,-8.261,3.360000,2.895,-8.261,3.360000,2.895,-8.261,3.360000,2.895,-8.261,3.360000,2.895,-8.261,3.360000,2.895
3,1SCL_A_4,U,4,-8.788,-1.854000,1.349,-8.788,-1.854000,1.349,-8.788,-1.854000,1.349,-8.788,-1.854000,1.349,-8.788,-1.854000,1.349
4,1SCL_A_5,G,5,-7.089,-6.228000,-0.914,-7.089,-6.228000,-0.914,-7.089,-6.228000,-0.914,-7.089,-6.228000,-0.914,-7.089,-6.228000,-0.914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,8Z1F_T_82,U,82,4.006,-64.322998,7.390,4.006,-64.322998,7.390,4.006,-64.322998,7.390,4.006,-64.322998,7.390,4.006,-64.322998,7.390
82,8Z1F_T_83,C,83,-0.629,-66.162003,9.502,-0.629,-66.162003,9.502,-0.629,-66.162003,9.502,-0.629,-66.162003,9.502,-0.629,-66.162003,9.502
83,8Z1F_T_84,A,84,-5.433,-68.873001,9.121,-5.433,-68.873001,9.121,-5.433,-68.873001,9.121,-5.433,-68.873001,9.121,-5.433,-68.873001,9.121
84,8Z1F_T_85,U,85,-9.027,-72.446999,7.096,-9.027,-72.446999,7.096,-9.027,-72.446999,7.096,-9.027,-72.446999,7.096,-9.027,-72.446999,7.096


In [23]:
submit_rhofold.columns

Index(['ID', 'resname', 'resid', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2',
       'x_3', 'y_3', 'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5'],
      dtype='object')

In [24]:
train_labels

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,ID,resname,resid,x_1,y_1,z_1
0,1SCL_A_1,G,1,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,1.827,-20.136000,11.793
...,...,...,...,...,...,...
137090,8Z1F_T_82,U,82,NaN,NaN,NaN
137091,8Z1F_T_83,C,83,NaN,NaN,NaN
137092,8Z1F_T_84,A,84,NaN,NaN,NaN
137093,8Z1F_T_85,U,85,NaN,NaN,NaN


In [25]:
train_labels.rename(columns={"x_1":"x_1_true","y_1":"y_1_true", "z_1":"z_1_true"}, inplace=True)

In [26]:
merged_df = submit_rhofold.merge(train_labels[["ID", "x_1_true", "y_1_true", "z_1_true"]], on="ID")

In [27]:
merged_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_1_true,y_1_true,z_1_true
0,1SCL_A_1,G,1,-2.356,12.116000,0.453,-2.356,12.116000,0.453,-2.356,...,0.453,-2.356,12.116000,0.453,-2.356,12.116000,0.453,13.760,-25.974001,0.102
1,1SCL_A_2,G,2,-5.678,8.330000,2.603,-5.678,8.330000,2.603,-5.678,...,2.603,-5.678,8.330000,2.603,-5.678,8.330000,2.603,9.310,-29.638000,2.669
2,1SCL_A_3,G,3,-8.261,3.360000,2.895,-8.261,3.360000,2.895,-8.261,...,2.895,-8.261,3.360000,2.895,-8.261,3.360000,2.895,5.529,-27.813000,5.878
3,1SCL_A_4,U,4,-8.788,-1.854000,1.349,-8.788,-1.854000,1.349,-8.788,...,1.349,-8.788,-1.854000,1.349,-8.788,-1.854000,1.349,2.678,-24.900999,9.793
4,1SCL_A_5,G,5,-7.089,-6.228000,-0.914,-7.089,-6.228000,-0.914,-7.089,...,-0.914,-7.089,-6.228000,-0.914,-7.089,-6.228000,-0.914,1.827,-20.136000,11.793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47852,8Z1F_T_82,U,82,4.006,-64.322998,7.390,4.006,-64.322998,7.390,4.006,...,7.390,4.006,-64.322998,7.390,4.006,-64.322998,7.390,NaN,NaN,NaN
47853,8Z1F_T_83,C,83,-0.629,-66.162003,9.502,-0.629,-66.162003,9.502,-0.629,...,9.502,-0.629,-66.162003,9.502,-0.629,-66.162003,9.502,NaN,NaN,NaN
47854,8Z1F_T_84,A,84,-5.433,-68.873001,9.121,-5.433,-68.873001,9.121,-5.433,...,9.121,-5.433,-68.873001,9.121,-5.433,-68.873001,9.121,NaN,NaN,NaN
47855,8Z1F_T_85,U,85,-9.027,-72.446999,7.096,-9.027,-72.446999,7.096,-9.027,...,7.096,-9.027,-72.446999,7.096,-9.027,-72.446999,7.096,NaN,NaN,NaN


In [28]:
merged_df.to_csv("rhofold_01_oof.csv")